In [ ]:
import phoebe
import numpy as np
import matplotlib.pyplot as plt
import emcee
import sys
%matplotlib inline

In [ ]:
b = phoebe.default_binary(contact_binary = True)

In [ ]:
#b.add_constraint('semidetached', 'primary')

In [ ]:
#b.add_constraint('semidetached', 'secondary')

In [ ]:
b['period@orbit'] = 1
b['sma@orbit'] = 1
#b['q@binary'] = 0.44
# b['incl@orbit'] = 83.5
# b['requiv@primary'] = 1.2
# b['requiv@secondary'] = 0.8
#b['teff@primary'] = 5660.
# b['teff@secondary'] = 5500.

In [ ]:
lc = np.loadtxt('data.lc')
#v737 = np.loadtxt('v737per.B')
#vdata = v737[:,1]
#flux = vdata/(-2.5)
#flux10 = 10**flux
#lc = v737
#lc[:,1] = flux10

In [ ]:
b.add_dataset('lc', times=lc[:,0], fluxes=lc[:,1], sigmas=0.05*np.ones(len(lc)), passband='Johnson:B')

In [ ]:
phoebe.interactive_checks_off()
phoebe.interactive_constraints_off()
b.set_value_all('irrad_method', 'none')

In [ ]:
b.flip_constraint('compute_phases', 'compute_times')
b['compute_phases@lc@dataset'] = np.linspace(-0.5,0.5,len(lc))

In [ ]:
def lnprob(x, adjpars, priors):
    #Check to see that all values are within the allowed limits:
    if not np.all([priors[i][0] < x[i] < priors[i][1] for i in range(len(priors))]):
         return -np.inf

    for i in range(len(adjpars)):
        b[adjpars[i]] = x[i]
    
    # Let's assume that our priors are uniform on the range of the physical parameter combinations.
    # This is already handled in Phoebe, which will throw an error if the system is not physical,
    # therefore it's easy to implement the lnprior as =0 when system checks pass and =-inf if they don't.
    # Here we'll 'package' this in a simple try/except statement:
    
    try:
        b.run_compute(irrad_method='none')

        # sum of squares of the residuals
        fluxes_model = b['fluxes@model'].interp_value(times=lc[:,0])
        lnp = -0.5*np.sum((fluxes_model-b['value@fluxes@dataset'])**2 / b['value@sigmas@dataset']**2) 
        
        #print('inp is ok!')

    except:
        lnp = -np.inf

    sys.stderr.write("lnp = %e\n" % (lnp))

    return lnp

In [ ]:
from scipy.stats import norm
def run(adjpars, priors, nwalkers, niter):
    ndim = len(adjpars)

   

    p0 = np.array([[p[0] + (p[1]-p[0])*np.random.rand() for p in priors] for i in range(nwalkers)])

#     pool = MPIPool()
#     if not pool.is_master():
#         pool.wait()
#         sys.exit(0)

    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=[adjpars, priors])

    pos, prob, state = sampler.run_mcmc(p0, niter)
    
    print(p0)
    
    for i in range(ndim):
        plt.figure()
        y = sampler.flatchain[:,i]
        n, bins, patches = plt.hist(y, 200, density=1, color="b", alpha=0.45)
        plt.title("Dimension {0:d}".format(i))
        
        mu = np.average(y)
        sigma = np.std(y)       
        print ("mu,", "sigma = ", mu, sigma)

        bf = norm.pdf(bins, mu, sigma)
        l = plt.plot(bins, bf, 'k--', linewidth=2.0)

    #plt.show()
    
    return pos

#     pool.close()



In [2]:
adjpars = ['requiv@primary', 'incl@orbit', 'q@binary', 'teff@primary', 'teff@secondary']
priors = [(0.45,0.55), (75.0, 80.0), (0.3, 0.5), (5400, 5800), (5400., 5800.)]
nwalkers = 20
niters = 2000
state = None

import time

time1 = time.time()
postion = run(adjpars, priors, nwalkers, niters)
time2 = time.time()

NameError: name 'run' is not defined

In [ ]:
from matplotlib.pyplot import cm 

mod = b
position = postion
times = lc[:,0]
color=cm.rainbow(np.linspace(0,1,nwalkers))

for i,c in zip(range(nwalkers),color):
    
    mod['requiv@primary'] = position[-1-i,0]
    mod['incl@binary@orbit@component'] = position[-1-i,1]
    mod['q@binary'] = position[-1-i,2]
    mod['teff@primary'] = position[-1-i,3]
    mod['teff@secondary'] = position[-1-i,4]
    try:
        mod.run_compute(model='run{}'.format(i))
    except:
        print('it is error')


for i,c in zip(range(nwalkers),color):
    try:
        model = mod['fluxes@run{}'.format(i)].interp_value(times=times)

    #plt.figure(1)
        plt.plot(times,model,c=c)
    except:
        print('it is error')
    
plt.plot(times,lc[:,1],"k.")
plt.xlabel("phases")
plt.ylabel("Flux")


In [ ]:
mod['incl@binary@orbit@component']

In [ ]:
mod['q@binary']

In [ ]:
mod['teff@primary']

In [ ]:
mod['teff@secondary']